<a href="https://colab.research.google.com/github/Ndey-Sambou/Guess-my-number/blob/master/DE_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# ======== API CONFIG (REPLACE THESE!) ========
EXCHANGE_API_KEY = "1e4a5a207d2743f034b711fa"       # From ExchangeRate-API (or leave blank for Frankfurter)
ALPHA_VANTAGE_API_KEY = "XIFV0Y7PVGHFVYPN" # From https://www.alphavantage.co/
WORLD_BANK_COUNTRY = "US"                       # ISO3 country code (e.g., "US", "CN")
WORLD_BANK_INDICATOR = "NY.GDP.MKTP.CD"         # GDP in USD (see https://data.worldbank.org/indicator)
STOCK_SYMBOL = "IBM"                            # Stock or crypto symbol (e.g., "BTC", "TSLA")

# ======== 1. WORLD BANK API (ECONOMIC DATA) ========
def fetch_world_bank_data(country, indicator, year="2022"):
    url = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}?date={year}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1 and data[1]:
            value = data[1][0]['value']
            return {
                "indicator": data[1][0]['indicator']['value'],
                "country": data[1][0]['country']['value'],
                "year": year,
                "value": f"${value:,.2f}" if indicator == "NY.GDP.MKTP.CD" else value
            }
    return {"error": "Failed to fetch World Bank data"}

# ======== 2. EXCHANGE RATE API ========
def fetch_exchange_rates(api_key=None):
    try:
        if api_key:  # Use ExchangeRate-API if key provided
            url = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/USD"
            rates = requests.get(url).json().get("conversion_rates", {})
        else:  # Fallback to Frankfurter (free)
            url = "https://api.frankfurter.app/latest?from=USD"
            rates = requests.get(url).json().get("rates", {})
        return rates if rates else {"error": "No exchange rate data"}
    except Exception as e:
        return {"error": f"Exchange API error: {str(e)}"}

# ======== 3. ALPHA VANTAGE API (STOCK/CRYPTO) ========
def fetch_alpha_vantage(symbol, api_key, function="TIME_SERIES_DAILY"):
    url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    if "Error Message" in data:
        return {"error": data["Error Message"]}
    return data

# ======== EXECUTE ALL APIS ========
if __name__ == "__main__":
    print("=== 🌍 WORLD BANK DATA ===")
    world_bank_data = fetch_world_bank_data(WORLD_BANK_COUNTRY, WORLD_BANK_INDICATOR)
    if "error" not in world_bank_data:
        print(f"{world_bank_data['country']} {world_bank_data['indicator']} ({world_bank_data['year']}): {world_bank_data['value']}")
    else:
        print(world_bank_data["error"])

    print("\n=== 💱 EXCHANGE RATES (USD BASE) ===")
    exchange_rates = fetch_exchange_rates(EXCHANGE_API_KEY)
    if "error" not in exchange_rates:
        df_rates = pd.DataFrame.from_dict(exchange_rates, orient="index", columns=["Rate"])
        display(df_rates.head(5))  # Show top 5 currencies
    else:
        print(exchange_rates["error"])

    print("\n=== 📈 ALPHA VANTAGE STOCK/CRYPTO DATA ===")
    stock_data = fetch_alpha_vantage(STOCK_SYMBOL, ALPHA_VANTAGE_API_KEY)
    if "error" not in stock_data:
        if "Time Series (Daily)" in stock_data:  # Stocks
            df_stock = pd.DataFrame.from_dict(stock_data["Time Series (Daily)"], orient="index")
            df_stock = df_stock.astype(float)
            print(f"Latest {STOCK_SYMBOL} stock data:")
            display(df_stock.head())
        elif "Time Series (Digital Currency Daily)" in stock_data:  # Crypto
            df_crypto = pd.DataFrame.from_dict(stock_data["Time Series (Digital Currency Daily)"], orient="index")
            print(f"Latest {STOCK_SYMBOL} crypto data:")
            display(df_crypto.head())
    else:
        print(stock_data["error"])

=== 🌍 WORLD BANK DATA ===
United States GDP (current US$) (2022): $26,006,893,000,000.00

=== 💱 EXCHANGE RATES (USD BASE) ===


,Rate
USD,1.0000
AED,3.6725
AFN,69.0154
ALL,83.1701
AMD,383.8891



=== 📈 ALPHA VANTAGE STOCK/CRYPTO DATA ===
Latest IBM stock data:


,1. open,2. high,3. low,4. close,5. volume
2025-07-23,284.30,288.0800,281.44,282.01,8105906.0
2025-07-22,284.74,284.8800,281.25,281.96,4824219.0
2025-07-21,286.29,287.7300,284.38,284.71,3051791.0
2025-07-18,283.38,287.1600,282.22,285.87,4478165.0
2025-07-17,281.50,283.4566,280.90,282.00,3337168.0
